In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("input"))

['test.csv', 'sample_submission.csv', 'train_images.zip', 'test_images.zip', 'train.csv', 'test_images', 'train_images']


In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')


In [3]:
from tqdm import tqdm
from PIL import Image

In [4]:
#the func is from https://www.kaggle.com/toshik/image-size-and-rate-of-new-whale
def get_size_list(targets, dir_target):

    result = list()

    for target in tqdm(targets):

        img = np.array(Image.open(os.path.join(dir_target, target+'.png')))
        result.append(str(img.shape))

    return result

In [6]:
train['size_info'] = get_size_list(train.id_code.tolist(), dir_target='input/train_images')


100%|██████████| 3662/3662 [04:54<00:00, 12.54it/s]


In [8]:
test['size_info'] = get_size_list(test.id_code.tolist(), dir_target='input/test_images')

100%|██████████| 1928/1928 [00:50<00:00, 38.31it/s]


In [9]:
train.size_info.value_counts()


(1050, 1050, 3)    974
(1736, 2416, 3)    638
(1958, 2588, 3)    533
(2136, 3216, 3)    410
(1536, 2048, 3)    351
(614, 819, 3)      287
(2588, 3388, 3)    141
(1000, 1504, 3)     92
(1226, 1844, 3)     61
(2848, 4288, 3)     52
(480, 640, 3)       42
(1944, 2896, 3)     34
(1424, 2144, 3)     28
(1117, 1476, 3)     14
(358, 474, 3)        2
(1110, 1467, 3)      2
(1764, 2146, 3)      1
Name: size_info, dtype: int64

In [10]:
test.size_info.value_counts()


(480, 640, 3)      1403
(1736, 2416, 3)     225
(1958, 2588, 3)     134
(1050, 1050, 3)      69
(614, 819, 3)        45
(1536, 2048, 3)      28
(1944, 2896, 3)      11
(1944, 2592, 3)       6
(1117, 1476, 3)       2
(576, 768, 3)         2
(1110, 1467, 3)       2
(1764, 2146, 3)       1
Name: size_info, dtype: int64

In [11]:
train_size_info = train.size_info.value_counts().to_frame().reset_index()
test_size_info = test.size_info.value_counts().to_frame().reset_index()
size_info = train_size_info.merge(test_size_info,on='index',how='outer').fillna(0)
size_info.columns = ['size','size_train','size_test']

In [12]:
size_info

,size,size_train,size_test
0,"(1050, 1050, 3)",974.0,69.0
1,"(1736, 2416, 3)",638.0,225.0
2,"(1958, 2588, 3)",533.0,134.0
3,"(2136, 3216, 3)",410.0,0.0
4,"(1536, 2048, 3)",351.0,28.0
5,"(614, 819, 3)",287.0,45.0
6,"(2588, 3388, 3)",141.0,0.0
7,"(1000, 1504, 3)",92.0,0.0
8,"(1226, 1844, 3)",61.0,0.0
9,"(2848, 4288, 3)",52.0,0.0


In [13]:
tmp = train.groupby(['size_info','diagnosis']).size().unstack().fillna(0).reset_index()
train_size_info.columns = ['size_info','size']
tmp = tmp.merge(train_size_info,on='size_info',how='left')
tmp

,size_info,0,1,2,3,4,size
0,"(1000, 1504, 3)",0.0,2.0,63.0,9.0,18.0,92
1,"(1050, 1050, 3)",901.0,19.0,39.0,2.0,13.0,974
2,"(1110, 1467, 3)",0.0,0.0,2.0,0.0,0.0,2
3,"(1117, 1476, 3)",10.0,0.0,0.0,2.0,2.0,14
4,"(1226, 1844, 3)",0.0,13.0,28.0,1.0,19.0,61
5,"(1424, 2144, 3)",0.0,1.0,22.0,3.0,2.0,28
6,"(1536, 2048, 3)",351.0,0.0,0.0,0.0,0.0,351
7,"(1736, 2416, 3)",31.0,234.0,233.0,60.0,80.0,638
8,"(1764, 2146, 3)",0.0,0.0,0.0,1.0,0.0,1
9,"(1944, 2896, 3)",0.0,6.0,3.0,21.0,4.0,34


In [14]:
for i in range(5):
    tmp.iloc[:,i+1] = tmp.iloc[:,i+1]/tmp['size']
tmp

,size_info,0,1,2,3,4,size
0,"(1000, 1504, 3)",0.000000,0.021739,0.684783,0.097826,0.195652,92
1,"(1050, 1050, 3)",0.925051,0.019507,0.040041,0.002053,0.013347,974
2,"(1110, 1467, 3)",0.000000,0.000000,1.000000,0.000000,0.000000,2
3,"(1117, 1476, 3)",0.714286,0.000000,0.000000,0.142857,0.142857,14
4,"(1226, 1844, 3)",0.000000,0.213115,0.459016,0.016393,0.311475,61
5,"(1424, 2144, 3)",0.000000,0.035714,0.785714,0.107143,0.071429,28
6,"(1536, 2048, 3)",1.000000,0.000000,0.000000,0.000000,0.000000,351
7,"(1736, 2416, 3)",0.048589,0.366771,0.365204,0.094044,0.125392,638
8,"(1764, 2146, 3)",0.000000,0.000000,0.000000,1.000000,0.000000,1
9,"(1944, 2896, 3)",0.000000,0.176471,0.088235,0.617647,0.117647,34


In [15]:
train.diagnosis.value_counts().sort_index()/3662


0    0.492900
1    0.101038
2    0.272802
3    0.052703
4    0.080557
Name: diagnosis, dtype: float64